# Stock Predictions

In [75]:
# Cell 1: Import necessary libraries
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from tkinter import *
from tkinter import ttk


In [70]:
# Cell 2: Define functions for fetching and preparing historical data
def fetch_historical_data(symbol, api_key):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={api_key}'
    response = requests.get(url)
    data = response.json()
    if 'Time Series (Daily)' in data:
        return data['Time Series (Daily)']
    else:
        return None

def prepare_historical_data(historical_data):
    dates = []
    close_prices = []
    for date, values in historical_data.items():
        dates.append(date)
        close_prices.append(float(values['4. close']))
    return pd.Series(close_prices, index=pd.to_datetime(dates))


In [71]:
# Cell 3: Define functions for resampling data, training model, generating predictions, and plotting
def resample_data(historical_data, interval):
    return historical_data.resample(interval).agg({'4. close': 'ohlc'})

def train_model(X, y):
    model = LinearRegression()
    model.fit(X, y)
    return model

def generate_predictions(model, X):
    return model.predict(X)

def plot_predictions(predictions, title):
    plt.figure(figsize=(10, 6))
    plt.plot(predictions.index, predictions['actual'], label='Actual Close Price', color='blue')
    plt.plot(predictions.index, predictions['predicted'], label='Predicted Close Price', color='red', linestyle='--')
    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.legend()
    plt.show()


In [72]:
# Cell 4: Main function to interact with user interface
def main():
    def predict():
        interval = interval_var.get()
        api_key = api_key_entry.get()
        symbol = symbol_entry.get()

        historical_data = fetch_historical_data(symbol, api_key)
        if historical_data:
            historical_series = prepare_historical_data(historical_data)
            resampled_data = resample_data(historical_series, interval)

            X = np.arange(len(resampled_data)).reshape(-1, 1)
            y = resampled_data['close']

            model = train_model(X, y)
            predictions = generate_predictions(model, X)

            plot_predictions(pd.DataFrame({'actual': y, 'predicted': predictions}, index=resampled_data.index), f'Close Prices for {symbol}')

        else:
            messagebox.showerror("Error", "Unable to fetch historical data.")

    root = Tk()
    root.title("Stock Price Prediction")

    # API Key Entry
    api_key_label = Label(root, text="Enter API Key:")
    api_key_label.grid(row=0, column=0)
    api_key_entry = Entry(root)
    api_key_entry.grid(row=0, column=1)

    # Stock Symbol Entry
    symbol_label = Label(root, text="Enter Stock Symbol:")
    symbol_label.grid(row=1, column=0)
    symbol_entry = Entry(root)
    symbol_entry.grid(row=1, column=1)

    # Interval Selection
    interval_label = Label(root, text="Select Prediction Interval:")
    interval_label.grid(row=2, column=0)
    interval_var = StringVar()
    interval_dropdown = ttk.Combobox(root, textvariable=interval_var)
    interval_dropdown['values'] = ('1D', '5D', '1W', '1M')
    interval_dropdown.grid(row=2, column=1)

    # Predict Button
    predict_button = Button(root, text="Predict", command=predict)
    predict_button.grid(row=3, column=0, columnspan=2)

    root.mainloop()

if __name__ == "__main__":
    main()
